In [ ]:
# NLP: IMDb 영화 리뷰 감성분석

In [ ]:
# 1. IMDb 영화 리뷰 데이터셋

# 라이브러리 설정
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import imdb
import json

# 랜덤 시드 고정
SEED = 12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 데이터셋 불러오기 및 크기 확인
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
# load_data(index_from=3) 값 추가시 디코딩 사전 인덱스가 3씩 더해짐
# 그리고 0, 1, 2 위치의 인덱스에는 <PAD>, <START>, <UNK> 가 자동으로 생성되지만 3 위치의 <UNUSED> 는 생성되지 않음, 필요하다면 수동으로 추가해줘야함
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")

# 첫번째 리뷰 값(정수) 확인 + 길이 확인
print(f"\n첫번째 리뷰 벡터 값들:\n{X_train[0]}")
print(f"첫번째 리뷰 길이(단어 개수): {len(X_train[0])}\n")

# 디코딩 값 (딕셔너리 형태) [:10] 출력
word_index = imdb.get_word_index()
# for k, v in list(word_index.items())[:10]:
#   print(f"{k}: {v}")
print(json.dumps(dict(list(word_index.items())[:10]), indent=2, ensure_ascii=False))

# 케라스 사전 전처리 및 벡터를 텍스트로 변환
word_index = {k: (v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

def decode_review(text_ids):
  index_to_word = {v: i for i, v in word_index.items()}
  return ' '.join([index_to_word.get(i, '?') for i in text_ids])

# 첫번째 리뷰 디코딩
encoded_review = X_train[0]
decoded_review = decode_review(encoded_review)
print(f"\n첫번째 리뷰 디코딩:\n{decoded_review}")

# 첫번째 리뷰 정답 레이블
print(f"첫번째 리뷰 정답 레이블: {y_train[0]}")

# 각 리뷰의 단어 개수 분포
review_length = [len(review) for review in X_train]
sns.displot(review_length)
plt.show()

In [ ]:
# 2. zero padding

from tensorflow.keras.preprocessing import sequence

X_train_pad = sequence.pad_sequences(X_train, maxlen=250)
X_test_pad = sequence.pad_sequences(X_test, maxlen=250)

print(X_train_pad[0])

In [ ]:
# 3. Word Embedding

# Embedding: 단어 사이의 관계를 학습해서 서로 유사한 단어들끼리 벡터 공간상에서 비슷한 위치에 배치

# 이는 희소한 One-Hot Encoding과 달리, 단어들을 연속적인 밀집(dense) 벡터 공간에 표현하는 방식

# *희소하다: 대부분의 값이 0이고, 실제로 의미있는 값(0이 아닌 값)은 거의 없다는 것을 의미

# 모델 정의

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU
import matplotlib.pyplot as plt

model_types = [SimpleRNN, LSTM, GRU]
def build_model(model_type):
  print(f"{model_type.__name__} 모델 학습중...")
  model = Sequential()
  
  # Embedding
  model.add(Embedding(input_dim=10000, output_dim=128, input_shape=(250,)))
  
  # model_type
  model.add(model_type(64, return_sequences=True)) 
  model.add(model_type(64))
  # return_sequences 값이 필요한 이유는 해당 파라미터의 기본값이 False 이고, 그 경우 timesteps(시간축) 차원이 사라진 2차원 형태로 값을 반환하기 때문에 더이상 3차원 input 형태를 요구하는 RNN 모델에 넣을 수 없게됨
  
  # Dense Classifier
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  
  # Compile
  model.compile(
		optimizer='adam',
		loss='binary_crossentropy',
		metrics=['accuracy']
	)
  model.summary()
  
  # fit (verbose = 2: epochs 단위 로그)
  rnn_history = model.fit(
		X_train_pad, y_train,
		epochs = 10,
		batch_size = 32,
		validation_split = 0.1,
		verbose = 2
	)
  
  return model, rnn_history

for model_type in model_types:
  _, rnn_history = build_model(model_type)
  
  def plot_metrics(history, model_type):
    fig, axes = plt.subplots(1, 2, figsize=(10,5))
    
    # Loss: 손실 함수
    axes[0].plot(range(1, 11), history.history['loss'][:10], label='Train')
    axes[0].plot(range(1, 11), history.history['val_loss'][:10], label='Validation')
    axes[0].set_title(f"{model_type.__name__} Loss")
    axes[0].legend()
    
    # Accuracy: 예측 정확도
    axes[1].plot(range(1, 11), history.history['accuracy'][:10], label='Train')
    axes[1].plot(range(1, 11), history.history['val_accuracy'][:10], label='Validation')
    axes[1].set_title(f"{model_type.__name__} Accuracy")
    axes[1].legend()
    
    plt.show()
  
  # 그래프 그리기
  plot_metrics(rnn_history, model_type)
    